# Progress Metering for Biofuels

In [2]:
import os
import re
from collections import defaultdict, OrderedDict
import pandas as pd
import numpy as np

# If you're using this for new results, simply set your default directory and you're good to go

In [13]:
directory = '/gss_gpfs_scratch/harms.n/bioTST/biofuels_rerun/'
results = defaultdict(OrderedDict)
gotPast6 = []
gotPast9 = []
for i in range(1,697):
    r = results[i]
    filename = 'AutoTST-biofuels.{0:d}.combined.log'.format(i)
    #filename = 'AutoTST-biofuels.xxx.combined.log'
    filepath = os.path.join(directory,filename)
    if os.path.exists(filepath):
        r['1 log file exists'] = 1
    else:
        continue
    with open(filepath) as f:
        lines = f.readlines()
    for j,l in enumerate(lines):
        #m = re.match('biofuelsTST.py:105 <module> INFO (.*)', l)
        if 'biofuelsSymmetryReRun.py:97' in l:
            m = l.split(' ')[-1][:-1]
            r['0 reaction'] = m
        if 'We have generated a H_Abstraction reaction that matches, and used it to label the atoms' in l:
            r['2 matched H-abstraction'] = 1
        if 'Generating a TS geometry via the direct guess method' in l:
            r['3 started making TS geometry'] = 1
        if 'Reading existing ts file' in l:
            r['4 using existing TS data file'] = 1
        if 'Symmetry input file written to' in l:
            r['5 starting Symmetry calculation'] = 1
        if 'Point group:' in l:
            r['6 Symmetry calc successful'] = 1
            gotPast6.append(str(filepath))
        """if "Running symmetry on the point group calculation has failed." in l:
            filename = 'AutoTST-biofuels-rerun.{0:d}.combined.log'.format(i)
            filepath = os.path.join(directory,filename)
            if os.path.exists(filepath):
                continue"""
        if 'line 295, in saveCoordinatesFromRDMol' in l:
            r['7 saveCoordinatesFromRDMol bug'] = -1
            print ''.join(lines[j-30:])
        if 'CanTherm execution initiated' in l:
            r['9 CanTherm started'] = 1
            gotPast9.append(str(filepath))
        if 'One or both of the barrier heights of' in l:
            r['9a CanTherm barrier height problem'] = -1
        if "Reading existing kinetics file" in l:
            r['A using prior calculation result'] = 1
        if 'Yay, reaction kinetics calculated!!!' in l:
            r['B overall success'] = 1

In [14]:
df = pd.DataFrame(results)
df

,1,2,3,4,5,6,7,8,9,10,...,687,688,689,690,691,692,693,694,695,696
1 log file exists,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
2 matched H-abstraction,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
3 started making TS geometry,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,NaN,...,1.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4 using existing TS data file,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5 starting Symmetry calculation,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6 Symmetry calc successful,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9 CanTherm started,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9a CanTherm barrier height problem,NaN,-1.0,-1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A using prior calculation result,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,...,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN
B overall success,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,...,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN


In [11]:
i = 0
for j,l in enumerate(lines):
    l = str(l)
    print l
    if 'biofuelsTST.py:105 <module> INFO' in l:
        print "yay"
        i += 1
    else:
        print "boo"
        

biofuelsTST.py:40 <module> INFO RUNNING WITH JOB NUMBER i = 692

boo
biofuelsTST.py:44 <module> INFO Loading RMG Database ...

boo
biofuelsTST.py:47 <module> INFO /home/harms.n/Code/RMG-database/input

boo
thermo.py:760 loadLibraries INFO Loading thermodynamics library from thermo_DFT_CCSDTF12_BAC.py in /home/harms.n/Code/RMG-database/input/thermo/libraries...

boo
thermo.py:760 loadLibraries INFO Loading thermodynamics library from CBS_QB3_1dHR.py in /home/harms.n/Code/RMG-database/input/thermo/libraries...

boo
thermo.py:760 loadLibraries INFO Loading thermodynamics library from primaryThermoLibrary.py in /home/harms.n/Code/RMG-database/input/thermo/libraries...

boo
thermo.py:760 loadLibraries INFO Loading thermodynamics library from KlippensteinH2O2.py in /home/harms.n/Code/RMG-database/input/thermo/libraries...

boo
thermo.py:774 loadGroups INFO Loading thermodynamics group database from /home/harms.n/Code/RMG-database/input/thermo/groups...

boo
transport.py:294 loadGroups INFO L

In [15]:
df.sum(axis=1)

1 log file exists                     692.0
2 matched H-abstraction               692.0
3 started making TS geometry          574.0
4 using existing TS data file          84.0
5 starting Symmetry calculation        84.0
6 Symmetry calc successful             84.0
9 CanTherm started                     70.0
9a CanTherm barrier height problem    -16.0
A using prior calculation result      118.0
B overall success                     161.0
dtype: float64

In [6]:
df.count(axis=1)

0 reaction                            72
1 log file exists                     72
2 matched H-abstraction               72
3 started making TS geometry          64
4 using existing TS data file         18
5 starting Symmetry calculation       18
6 Symmetry calc successful            18
9 CanTherm started                    11
9a CanTherm barrier height problem     2
A using prior calculation result       8
B overall success                     17
dtype: int64

In [7]:
for i in range(len(df.columns)):
    if np.isnan(df.iloc[-1,i]) == False:
        print df.iloc[-1,i]
        print df.iloc[0,i]

1
OO(15)+C8H17(18)=C8H18(2)+OOH(1)
1
OO(15)+C8H17(19)=C8H18(2)+OOH(1)
1
OO(15)+C8H17(24)=C8H18(3)+OOH(1)
1
OO(15)+C8H17(25)=C8H18(3)+OOH(1)
1
OO(15)+C8H17(26)=C8H18(3)+OOH(1)
1
C8H18(4)+OOH(1)=OO(15)+C8H17(28)
1
OO(15)+C8H17(29)=C8H18(4)+OOH(1)
1
OO(15)+C8H17(30)=C8H18(4)+OOH(1)
1
OO(15)+C8H17(32)=C8H18(4)+OOH(1)
1
OO(15)+C8H17(33)=C8H18(4)+OOH(1)
1
OO(15)+C6H13O(41)=C6H14O(6)+OOH(1)
1
OO(15)+C6H13O(42)=C6H14O(6)+OOH(1)
1
OO(15)+C6H13O(43)=C6H14O(6)+OOH(1)
1
OO(15)+C6H13O(44)=C6H14O(6)+OOH(1)
1
OO(15)+C6H13O(46)=C6H14O(6)+OOH(1)
1
OO(15)+C6H11(54)=C6H12(8)+OOH(1)
1
OO(15)+C6H11(55)=C6H12(8)+OOH(1)


## This part is used to try and figure out why some log files didn't make it from step 6 to step 9 

In [8]:
df.loc['A using prior calculation result'][df.loc['3 started making TS geometry']!=1]

13    1
14    1
15    1
17    1
18    1
31    1
39    1
40    1
Name: A using prior calculation result, dtype: object

The eight that did NOT have step 3 "started making TS geometry" all had "A using prior calculation result" i.e. they had worked, on a previous iteration. For example:

In [9]:
df[13]

0 reaction                            C8H18(4)+OOH(1)=OO(15)+C8H17(28)
1 log file exists                                                    1
2 matched H-abstraction                                              1
3 started making TS geometry                                       NaN
4 using existing TS data file                                      NaN
5 starting Symmetry calculation                                    NaN
6 Symmetry calc successful                                         NaN
9 CanTherm started                                                 NaN
9a CanTherm barrier height problem                                 NaN
A using prior calculation result                                     1
B overall success                                                    1
Name: 13, dtype: object

In [10]:
# Which had at least one successful symmetry but did not start cantherm
df.loc['0 reaction'][(df.loc['9 CanTherm started']!=1) & (df.loc['6 Symmetry calc successful']==1) ]

12      OO(15)+C8H17(27)=C8H18(3)+OOH(1)
19      OO(15)+C8H17(34)=C8H18(4)+OOH(1)
20      OO(15)+C8H17(35)=C8H18(4)+OOH(1)
21    OO(15)+C5H11O(36)=C5H12O(5)+OOH(1)
22    OO(15)+C5H11O(37)=C5H12O(5)+OOH(1)
23    OO(15)+C5H11O(38)=C5H12O(5)+OOH(1)
24    OO(15)+C5H11O(39)=C5H12O(5)+OOH(1)
Name: 0 reaction, dtype: object

* 12 weird cclib parsing error. suggest we try again
* 19 looks like it got interrupted on one of the molecules: [CH2]CCCC(C)CC. no errors reported, just stopped
* 20 looks like it got interrupted on one of the molecules: [CH2]CC(C)CCCC. no errors reported, just stopped
* 21 looks like it got interrupted on one of the molecules: CCC(C)(C)O
* 22 looks like it got interrupted on one of the molecules: CCC(C)(C)O
* 23 looks like it got interrupted on one of the molecules: CCC(C)(C)O. race condition with 22?
* 24 looks like it got interrupted on one of the molecules: CCC(C)(C)[O]

In [11]:
gotPast6

['./results/AutoTST-biofuels-rerun.1.combined.log',
 './results/AutoTST-biofuels-rerun.1.combined.log',
 './results/AutoTST-biofuels-rerun.1.combined.log',
 './results/AutoTST-biofuels-rerun.1.combined.log',
 './results/AutoTST-biofuels-rerun.3.combined.log',
 './results/AutoTST-biofuels-rerun.3.combined.log',
 './results/AutoTST-biofuels-rerun.3.combined.log',
 './results/AutoTST-biofuels-rerun.3.combined.log',
 './results/AutoTST-biofuels-rerun.4.combined.log',
 './results/AutoTST-biofuels-rerun.4.combined.log',
 './results/AutoTST-biofuels-rerun.4.combined.log',
 './results/AutoTST-biofuels-rerun.4.combined.log',
 './results/AutoTST-biofuels-rerun.9.combined.log',
 './results/AutoTST-biofuels-rerun.9.combined.log',
 './results/AutoTST-biofuels-rerun.9.combined.log',
 './results/AutoTST-biofuels-rerun.9.combined.log',
 './results/AutoTST-biofuels-rerun.10.combined.log',
 './results/AutoTST-biofuels-rerun.10.combined.log',
 './results/AutoTST-biofuels-rerun.10.combined.log',
 './resul

In [12]:
gotPast9

['./results/AutoTST-biofuels-rerun.1.combined.log',
 './results/AutoTST-biofuels-rerun.3.combined.log',
 './results/AutoTST-biofuels-rerun.4.combined.log',
 './results/AutoTST-biofuels-rerun.9.combined.log',
 './results/AutoTST-biofuels-rerun.10.combined.log',
 './results/AutoTST-biofuels-rerun.11.combined.log',
 './results/AutoTST-biofuels-rerun.26.combined.log',
 './results/AutoTST-biofuels-rerun.27.combined.log',
 './results/AutoTST-biofuels-rerun.28.combined.log',
 './results/AutoTST-biofuels-rerun.29.combined.log',
 './results/AutoTST-biofuels-rerun.30.combined.log']